In [1]:
import polaris
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

from qsarmil.lazy import LazyMIL
from qsarcons.consensus import RandomSearchRegressor, SystematicSearchRegressor, GeneticSearchRegressor

/storage/dmitry/miniforge3/envs/tmp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 1. Load data

In [2]:
# Load the benchmark from the Hub
benchmark = polaris.load_benchmark("polaris/adme-fang-solu-1")

# Get the train and test data-loaders
data_train, data_test = benchmark.get_train_test_split()
data_train, data_test = data_train.as_dataframe(), data_test.as_dataframe()

smi_train, prop_train = data_train["smiles"].to_list(), data_train["LOG_SOLUBILITY"].to_list()

data_train, data_val = train_test_split(data_train, test_size=0.2, random_state=42)

[2025-10-09 07:49:48] INFO     The version of Polaris that was used to create the artifact (0.0.0)  ]8;id=234053;file:///storage/dmitry/miniforge3/envs/tmp/lib/python3.10/site-packages/polaris/_artifact.py\_artifact.py]8;;\:]8;id=146316;file:///storage/dmitry/miniforge3/envs/tmp/lib/python3.10/site-packages/polaris/_artifact.py#96\96]8;;\
                               is different from the currently installed version of Polaris                        
                               (0.13.0).                                                                           

                      INFO     To verify the checksum, we need to recompute it. This can be slow    ]8;id=91161;file:///storage/dmitry/miniforge3/envs/tmp/lib/python3.10/site-packages/polaris/mixins/_checksum.py\_checksum.py]8;;\:]8;id=619176;file:///storage/dmitry/miniforge3/envs/tmp/lib/python3.10/site-packages/polaris/mixins/_checksum.py#67\67]8;;\
                               for large datasets.                                                                 

[07:49:48]  Success: Fetching dataset                                                                 ]8;id=529903;file:///storage/dmitry/miniforge3/envs/tmp/lib/python3.10/site-packages/polaris/utils/context.py\context.py]8;;\:]8;id=631262;file:///storage/dmitry/miniforge3/envs/tmp/lib/python3.10/site-packages/polaris/utils/context.py#53\53]8;;\

                      INFO     The version of Polaris that was used to create the artifact (0.0.0)  ]8;id=681453;file:///storage/dmitry/miniforge3/envs/tmp/lib/python3.10/site-packages/polaris/_artifact.py\_artifact.py]8;;\:]8;id=735392;file:///storage/dmitry/miniforge3/envs/tmp/lib/python3.10/site-packages/polaris/_artifact.py#96\96]8;;\
                               is different from the currently installed version of Polaris                        
                               (0.13.0).                                                                           

                      WARNING  No checksum to verify against. Specify either the md5sum parameter   ]8;id=291704;file:///storage/dmitry/miniforge3/envs/tmp/lib/python3.10/site-packages/polaris/mixins/_checksum.py\_checksum.py]8;;\:]8;id=848749;file:///storage/dmitry/miniforge3/envs/tmp/lib/python3.10/site-packages/polaris/mixins/_checksum.py#60\60]8;;\
                               or store the checksum in the dataset.md5sum attribute.                              

            Success: Fetching benchmark                                                               ]8;id=167414;file:///storage/dmitry/miniforge3/envs/tmp/lib/python3.10/site-packages/polaris/utils/context.py\context.py]8;;\:]8;id=732052;file:///storage/dmitry/miniforge3/envs/tmp/lib/python3.10/site-packages/polaris/utils/context.py#53\53]8;;\

### 2. Build multiple 2D models

In [4]:
data_test["LogS"] = [0 for i in data_test.index]

lazy_mil = LazyMIL(task="regression", hopt=False, output_folder="logs_bench", n_cpu=20, verbose=True)
lazy_mil.run(data_train, data_val, data_test)

Generating conformers: 100%|██████████████████████████████████████████████████████████| 400/400 [00:19<00:00, 20.70it/s]


72 / 72 / MolFeatPmapper|DynamicPoolingNetworkRegressorsoressoressor

### 3. Build model consensus

In [5]:
metric = "auto"
cons_size = "auto"

In [6]:
cons_methods = [
    ("Best", SystematicSearchRegressor(cons_size=1, metric=metric)),         
    ("Random", RandomSearchRegressor(cons_size=cons_size, n_iter=1000, metric=metric)),       
    ("Systematic", SystematicSearchRegressor(cons_size=cons_size, metric=metric)),
    ("Genetic", GeneticSearchRegressor(cons_size=cons_size, n_iter=50, pop_size=50, mut_prob=0.2, metric=metric))
]

In [7]:
# load model predictions
df_val = pd.read_csv("logs_bench/val.csv")
df_test = pd.read_csv("logs_bench/test.csv")

# skip first two columns (smiles and true property value)
x_val, true_val = df_val.iloc[:, 2:], df_val.iloc[:, 1]
x_test = df_test.iloc[:, 2:]

In [8]:
for name, cons_searcher in cons_methods:

    # run search
    best_cons = cons_searcher.run(x_val, true_val)
    
    # make val and test predictions
    pred_val = cons_searcher._consensus_predict(x_val[best_cons])
    pred_test = cons_searcher._consensus_predict(x_test[best_cons])
    
    # write prediction accuracy metric
    df_val[name] = pred_val
    df_test[name] = pred_test

### 4. Summurize results

In [9]:
res = pd.DataFrame()
for model in df_val.columns[2:]:
    res.loc[model, "R2"] = r2_score(df_val["Y_TRUE"], df_val[model])

In [10]:
res.sort_values(by="R2", ascending=False)

,R2
Genetic,0.401268
Random,0.377951
Systematic,0.372215
RDKitGETAWAY|DynamicPoolingNetworkRegressor,0.311625
Best,0.311625
...,...
MolFeatUSRD|DynamicPoolingNetworkRegressor,0.064956
RDKitWHIM|MeanInstanceWrapperMLPNetworkRegressor,0.010889
MolFeatUSRD|MeanInstanceWrapperMLPNetworkRegressor,-0.015881
MolFeatPmapper|MeanBagWrapperMLPNetworkRegressor,-0.027218


In [16]:
y_pred = df_test["Genetic"].to_list()
results = benchmark.evaluate(y_pred)
results

{
  "benchmark_artifact_id": "polaris/adme-fang-solu-1",
  "benchmark_name": null,
  "benchmark_owner": null,
  "results": [
    {
      "test_set": "test",
      "target_label": "LOG_SOLUBILITY",
      "scores": {
        "mean_squared_error": 0.378813039771392,
        "spearmanr": 0.42682041684077643,
        "explained_var": 0.30162512086422466,
        "pearsonr": 0.5527202404910281,
        "mean_absolute_error": 0.43516512725032,
        "r2": 0.3013132355128497
      }
    }
  ],
  "name": null,
  "description": "",
  "tags": [],
  "user_attributes": {},
  "owner": null,
  "polaris_version": "0.13.0",
  "slug": null,
  "github_url": null,
  "paper_url": null,
  "contributors": [],
  "artifact_id": null,
  "urn": null,
  "version": 1
}